# Reference design - E core inductor

**Authors** : S. Gaydier, I. Zehavi, T. Cherrière, P. Gangl

In [ ]:
from shapeOptInductor import gen_mesh9
import ngsolve as ngs
from ngsolve.webgui import Draw
import numpy as np


## 1 - Geometry and meshing

In [ ]:
airgap = 4.11e-3  # size of the airgap
lz = 1e-2  # thickness in the z-axis
maxh = 2e-3  # maximum element size
s = 4  # symmetry factor (we simulate only one quarter of the inductor)

# generate the geometry (cf shapOpt.py for details)
mesh = gen_mesh9(airgap, maxh)

# define the characteristic functions of all regions
XiAir = mesh.MaterialCF({"air": 1})
XiCore = mesh.MaterialCF({"core": 1})
XiCoil = mesh.MaterialCF({"coil": 1})

# draw the meshed geometry
Draw(1 * XiAir + 2 * XiCoil + 3 * XiCore, mesh, radius=0.02)


## 2 - Computation of (complex) magnetic state

### a) Parameters definition

In [ ]:
f = 5e4  # working frequency (Hz)
omega = 2 * np.pi * f  # rad/s
mu0 = 4e-7 * np.pi  # void permeability (H/m)
mur = 1000  # relative permeability of iron (no unit)
mu_iron = mur * mu0  # permeability of iron (H/m)
delta = 0.1  # loss angle associated with the coil (rad)
mu_coil = np.exp(-1j * delta) * mu0  # complex permeability
nb_turn = 200  # number of turn in the coil (no unit)
Is = 2  # source current intensity (A)
js = nb_turn / 2 * Is / (ngs.Integrate(XiCoil, mesh)) * XiCoil  # source current density (A/m²)


### b) Processing

We want to find $\underline{a}\in H^1(\Omega, \mathbb C)$ solving the following equality for any $a^* \in H^1(\Omega, \mathbb R)$, such that $\underline{a} = 0$ on the outer air box boundary and vertical symmetry axis.

$$ \int_\Omega \nabla a^* \cdot \left( \frac{\chi_c}{\underline{\mu_f}} + \frac{\chi_f}{\underline{\mu_f}} +\frac{\chi_a}{\underline{\mu_0}}\right) \nabla \underline{a} = \int_\Omega \chi_c j$$

In [ ]:
def magWeakFormComplex(a, a_):
    """Return the complex weak form of the magnetic problem, i.e, bilinear
    and linear forms (matrix and right-hand side after discretization, respectively)"""

    # bilinear form (transmission)
    bf = ngs.grad(a_) * 1 / mu_iron * ngs.grad(a) * ngs.dx("core")
    bf += ngs.grad(a_) * 1 / mu_coil * ngs.grad(a) * ngs.dx("coil")
    bf += ngs.grad(a_) * 1 / mu0 * ngs.grad(a) * ngs.dx("air")

    # linear form (source)
    lf = a_ * js * ngs.dx("coil")

    return bf, lf


def solveStateComplex(mesh):
    """Solve the complex magnetic state"""
    # definition of function space
    fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
    a, a_ = fes.TnT()

    # definition of weak form
    bf, f = magWeakFormComplex(a, a_)
    K, F = ngs.BilinearForm(fes), ngs.LinearForm(fes)
    K += bf
    F += f

    # assembly
    K.Assemble()
    F.Assemble()

    # solving
    gf = ngs.GridFunction(fes)
    Kinv = K.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
    gf.vec.data = Kinv * F.vec

    return gf


# we solve and plot for the given geometry, and plot the vector potential (equipotential are flux-lines).
a = solveStateComplex(mesh)
Draw(a, mesh, animate_complex=True)


### c) Post-processing

The AC losses in conductors are computed from the imaginary part of their permeability (or in this case, reluctivity).
The current is assumed to be sinus. In reality, it would rather be triangular, which adds harmonics that are neglected here. Since we assume the linearity of the problem linear, one needs to solve the system for each frequency with the corresponding current harmonics, and sum the losses (which is not done here for simplicity and speed).
We compute the losses from the imaginary part of the reluctivity (in the coil)

$$ P(\underline{a}) = s l_z \pi f \int_\Omega \Im \left( \frac{\chi_c}{\underline{\mu_c}} \right) |\nabla \underline{a}|^2 \; \mathrm{d} x, $$

and the inductance from the real part of the reluctivity (everywhere)

$$ L(\underline{a}) = \frac{s l_z}{I^2} \int_\Omega \Re \left( \frac{\chi_c}{\underline{\mu_f}} + \frac{\chi_f}{\underline{\mu_f}} +\frac{\chi_a}{\underline{\mu_0}}\right) |\nabla \underline{a}|^2 \; \mathrm{d} x. $$

In [ ]:
def Losses(a, mesh):
    rel = XiCoil / mu_coil
    return s * np.pi * f * lz * ngs.Integrate(rel.imag * ngs.Norm(ngs.grad(a)) ** 2, mesh)


def Inductance(a, mesh):
    rel = XiAir / mu0 + XiCoil / mu_coil + XiCore / mu_iron
    return s * lz / (Is**2) * ngs.Integrate(rel.real * ngs.Norm(ngs.grad(a)) ** 2, mesh)


print(f" P_AC = {Losses(a,mesh) :.2f} W")
print(f" L = {Inductance(a,mesh) * 1e3 :.2f} mH")
